# Сверточные нейронные сети

## 1. Реализовать СНН на MNIST

In [3]:
import os
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
import torch.nn.functional as F

BATCH_SIZE = 32
NUM_EPOCHS = 10
FOLDER = 'MNIST_data'

if not os.path.exists(FOLDER):
    os.mkdir(FOLDER)
    
trans = transforms.Compose([transforms.ToTensor()])

# данные
train_set = datasets.MNIST(root=FOLDER, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=FOLDER, train=False, transform=trans, download=True)

# итераторы
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=False)
sample_x, sample_y = next(iter(train_loader))

In [98]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.prelu1 = nn.PReLU()
        self.bn1 = nn.BatchNorm2d(6)
#        self.bn1 = CustomBatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.prelu2 = nn.PReLU()
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(self.bn1(self.prelu1(self.conv1(x))), (2,2))
        x = F.max_pool2d(self.bn2(self.prelu2(self.conv2(x))), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.1)
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = LeNet()
print (net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (prelu1): PReLU(num_parameters=1)
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (prelu2): PReLU(num_parameters=1)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [99]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [100]:
for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        out = net(x)
        loss = criterion(out, y)
        # оптимизация параметров
        ## первым шагом обнулим градиенты предыдущего шага (важный момент,
        ## без этого результаты теряют корректность)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # подсчет статистики за эпоху        
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))




  0%|          | 0/1875 [00:00<?, ?it/s]


  0%|          | 3/1875 [00:00<01:09, 27.08it/s]


  0%|          | 6/1875 [00:00<01:10, 26.39it/s]


  0%|          | 9/1875 [00:00<01:09, 26.68it/s]


  1%|          | 13/1875 [00:00<01:06, 27.97it/s]


  1%|          | 16/1875 [00:00<01:06, 27.98it/s]


  1%|          | 19/1875 [00:00<01:18, 23.69it/s]


  1%|          | 22/1875 [00:00<01:17, 23.90it/s]


  1%|▏         | 25/1875 [00:00<01:14, 24.93it/s]


  2%|▏         | 29/1875 [00:01<01:08, 26.94it/s]


  2%|▏         | 33/1875 [00:01<01:05, 28.33it/s]


  2%|▏         | 36/1875 [00:01<01:15, 24.29it/s]


  2%|▏         | 39/1875 [00:01<01:25, 21.59it/s]


  2%|▏         | 42/1875 [00:01<01:26, 21.30it/s]


  2%|▏         | 45/1875 [00:01<01:25, 21.33it/s]


  3%|▎         | 48/1875 [00:02<01:31, 20.00it/s]


  3%|▎         | 51/1875 [00:02<01:31, 19.84it/s]


  3%|▎         | 54/1875 [00:02<01:34, 19.24it/s]


  3%|▎         | 57/1875 [00:02<01:30, 20.14it/s]


  3%|▎         | 60/1

 46%|████▌     | 858/1875 [00:34<00:35, 28.48it/s]


 46%|████▌     | 861/1875 [00:35<00:35, 28.60it/s]


 46%|████▌     | 865/1875 [00:35<00:34, 29.61it/s]


 46%|████▋     | 868/1875 [00:35<00:34, 29.50it/s]


 46%|████▋     | 871/1875 [00:35<00:34, 29.51it/s]


 47%|████▋     | 874/1875 [00:35<00:34, 29.34it/s]


 47%|████▋     | 877/1875 [00:35<00:34, 29.32it/s]


 47%|████▋     | 880/1875 [00:35<00:35, 28.40it/s]


 47%|████▋     | 884/1875 [00:35<00:32, 30.22it/s]


 47%|████▋     | 889/1875 [00:35<00:29, 33.06it/s]


 48%|████▊     | 893/1875 [00:36<00:28, 34.49it/s]


 48%|████▊     | 897/1875 [00:36<00:28, 34.36it/s]


 48%|████▊     | 901/1875 [00:36<00:28, 33.83it/s]


 48%|████▊     | 905/1875 [00:36<00:30, 32.13it/s]


 48%|████▊     | 909/1875 [00:36<00:30, 31.41it/s]


 49%|████▊     | 913/1875 [00:36<00:32, 29.77it/s]


 49%|████▉     | 917/1875 [00:36<00:30, 31.09it/s]


 49%|████▉     | 921/1875 [00:36<00:31, 30.21it/s]


 49%|████▉     | 925/1875 [00:37<00:33, 28.64i

 99%|█████████▊| 1847/1875 [01:06<00:01, 26.49it/s]


 99%|█████████▊| 1850/1875 [01:06<00:00, 27.33it/s]


 99%|█████████▉| 1853/1875 [01:06<00:00, 25.40it/s]


 99%|█████████▉| 1857/1875 [01:06<00:00, 27.02it/s]


 99%|█████████▉| 1860/1875 [01:07<00:00, 27.78it/s]


 99%|█████████▉| 1864/1875 [01:07<00:00, 29.24it/s]


100%|█████████▉| 1867/1875 [01:07<00:00, 28.87it/s]


100%|█████████▉| 1870/1875 [01:07<00:00, 28.68it/s]


100%|█████████▉| 1873/1875 [01:07<00:00, 27.92it/s]


100%|██████████| 1875/1875 [01:07<00:00, 27.76it/s]


  0%|          | 0/1875 [00:00<?, ?it/s]


  0%|          | 3/1875 [00:00<01:07, 27.84it/s]

Epoch: 0; Accuracy train: 95.852%





  0%|          | 5/1875 [00:00<01:21, 23.02it/s]


  0%|          | 8/1875 [00:00<01:16, 24.47it/s]


  1%|          | 11/1875 [00:00<01:13, 25.19it/s]


  1%|          | 14/1875 [00:00<01:12, 25.68it/s]


  1%|          | 18/1875 [00:00<01:06, 27.97it/s]


  1%|          | 22/1875 [00:00<01:02, 29.52it/s]


  1%|▏         | 26/1875 [00:00<01:00, 30.54it/s]


  2%|▏         | 29/1875 [00:01<01:00, 30.36it/s]


  2%|▏         | 32/1875 [00:01<01:01, 29.74it/s]


  2%|▏         | 35/1875 [00:01<01:09, 26.50it/s]


  2%|▏         | 38/1875 [00:01<01:08, 26.96it/s]


  2%|▏         | 42/1875 [00:01<01:04, 28.42it/s]


  2%|▏         | 45/1875 [00:01<01:03, 28.83it/s]


  3%|▎         | 49/1875 [00:01<01:01, 29.78it/s]


  3%|▎         | 53/1875 [00:01<00:59, 30.57it/s]


  3%|▎         | 57/1875 [00:01<00:59, 30.35it/s]


  3%|▎         | 61/1875 [00:02<00:59, 30.72it/s]


  3%|▎         | 65/1875 [00:02<01:01, 29.30it/s]


  4%|▎         | 68/1875 [00:02<01:01, 29.15it/s]


  4%|▍     

...

Epoch: 8; Accuracy train: 98.357%





  0%|          | 6/1875 [00:00<01:03, 29.64it/s]


  0%|          | 8/1875 [00:00<01:15, 24.85it/s]


  1%|          | 11/1875 [00:00<01:16, 24.37it/s]


  1%|          | 15/1875 [00:00<01:11, 25.99it/s]


  1%|          | 18/1875 [00:00<01:12, 25.71it/s]


  1%|          | 21/1875 [00:00<01:10, 26.16it/s]


  1%|▏         | 24/1875 [00:00<01:14, 24.82it/s]


  1%|▏         | 27/1875 [00:01<01:29, 20.60it/s]


  2%|▏         | 30/1875 [00:01<01:26, 21.22it/s]


  2%|▏         | 33/1875 [00:01<01:26, 21.41it/s]


  2%|▏         | 37/1875 [00:01<01:19, 22.98it/s]


  2%|▏         | 40/1875 [00:01<01:23, 22.06it/s]


  2%|▏         | 44/1875 [00:01<01:13, 24.79it/s]


  3%|▎         | 47/1875 [00:01<01:12, 25.25it/s]


  3%|▎         | 50/1875 [00:02<01:22, 22.08it/s]


  3%|▎         | 53/1875 [00:02<01:26, 21.06it/s]


  3%|▎         | 56/1875 [00:02<01:19, 22.86it/s]


  3%|▎         | 59/1875 [00:02<01:29, 20.28it/s]


  3%|▎         | 62/1875 [00:02<01:28, 20.45it/s]


  3%|▎     

 42%|████▏     | 793/1875 [00:35<00:47, 22.67it/s]


 42%|████▏     | 796/1875 [00:35<00:45, 23.58it/s]


 43%|████▎     | 799/1875 [00:35<00:46, 23.35it/s]


 43%|████▎     | 802/1875 [00:36<00:45, 23.54it/s]


 43%|████▎     | 805/1875 [00:36<00:46, 23.17it/s]


 43%|████▎     | 808/1875 [00:36<00:48, 21.83it/s]


 43%|████▎     | 811/1875 [00:36<00:51, 20.67it/s]


 43%|████▎     | 814/1875 [00:36<00:47, 22.22it/s]


 44%|████▎     | 817/1875 [00:36<00:48, 21.97it/s]


 44%|████▎     | 820/1875 [00:36<00:49, 21.43it/s]


 44%|████▍     | 823/1875 [00:37<00:47, 22.16it/s]


 44%|████▍     | 826/1875 [00:37<00:44, 23.54it/s]


 44%|████▍     | 829/1875 [00:37<00:45, 23.22it/s]


 44%|████▍     | 832/1875 [00:37<00:44, 23.59it/s]


 45%|████▍     | 835/1875 [00:37<00:41, 24.90it/s]


 45%|████▍     | 838/1875 [00:37<00:42, 24.45it/s]


 45%|████▍     | 841/1875 [00:37<00:49, 20.95it/s]


 45%|████▌     | 844/1875 [00:38<00:48, 21.27it/s]


 45%|████▌     | 847/1875 [00:38<00:47, 21.85i

 88%|████████▊ | 1655/1875 [01:09<00:07, 28.01it/s]


 88%|████████▊ | 1659/1875 [01:09<00:07, 27.98it/s]


 89%|████████▊ | 1663/1875 [01:09<00:06, 30.68it/s]


 89%|████████▉ | 1668/1875 [01:09<00:06, 33.24it/s]


 89%|████████▉ | 1672/1875 [01:09<00:05, 34.56it/s]


 89%|████████▉ | 1676/1875 [01:09<00:05, 35.85it/s]


 90%|████████▉ | 1680/1875 [01:09<00:05, 36.25it/s]


 90%|████████▉ | 1684/1875 [01:09<00:05, 37.19it/s]


 90%|█████████ | 1689/1875 [01:10<00:04, 37.65it/s]


 90%|█████████ | 1693/1875 [01:10<00:04, 37.89it/s]


 91%|█████████ | 1698/1875 [01:10<00:04, 39.44it/s]


 91%|█████████ | 1703/1875 [01:10<00:04, 40.50it/s]


 91%|█████████ | 1708/1875 [01:10<00:04, 37.97it/s]


 91%|█████████▏| 1712/1875 [01:10<00:04, 38.09it/s]


 92%|█████████▏| 1716/1875 [01:10<00:04, 38.63it/s]


 92%|█████████▏| 1720/1875 [01:10<00:03, 39.01it/s]


 92%|█████████▏| 1725/1875 [01:10<00:03, 40.33it/s]


 92%|█████████▏| 1730/1875 [01:11<00:03, 38.94it/s]


 92%|█████████▏| 1734/1875 [

Epoch: 9; Accuracy train: 98.445%


In [101]:
av_loss = 0.
correct = 0.
for x, y in tqdm.tqdm(test_loader):
    # рассчитываем функцию потерь
    out = net(x)
    loss = criterion(out, y)
    # подсчет статистики за эпоху        
    pred = out.max(1, keepdim=True)[1]
    correct += pred.eq(y.view_as(pred)).sum().item()
    av_loss += loss.item()
print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(test_loader.dataset) * 100))




  0%|          | 0/313 [00:00<?, ?it/s]


  2%|▏         | 7/313 [00:00<00:04, 64.52it/s]


  5%|▌         | 17/313 [00:00<00:04, 71.61it/s]


  9%|▊         | 27/313 [00:00<00:03, 78.13it/s]


 12%|█▏        | 38/313 [00:00<00:03, 84.20it/s]


 16%|█▌        | 50/313 [00:00<00:02, 91.24it/s]


 19%|█▉        | 61/313 [00:00<00:02, 94.62it/s]


 23%|██▎       | 71/313 [00:00<00:02, 91.31it/s]


 26%|██▌       | 82/313 [00:00<00:02, 94.04it/s]


 29%|██▉       | 92/313 [00:00<00:02, 94.39it/s]


 33%|███▎      | 102/313 [00:01<00:02, 92.87it/s]


 36%|███▋      | 114/313 [00:01<00:02, 97.86it/s]


 40%|███▉      | 124/313 [00:01<00:01, 95.34it/s]


 43%|████▎     | 134/313 [00:01<00:01, 92.27it/s]


 46%|████▌     | 144/313 [00:01<00:02, 82.14it/s]


 49%|████▉     | 153/313 [00:01<00:02, 74.81it/s]


 53%|█████▎    | 165/313 [00:01<00:01, 83.15it/s]


 56%|█████▌    | 175/313 [00:01<00:01, 85.81it/s]


 59%|█████▉    | 184/313 [00:02<00:01, 80.86it/s]


 62%|██████▏   | 193/313 [00:

Epoch: 9; Accuracy train: 98.150%


# Custrom BatchNorm2d

In [102]:
class CustomBatchNorm2d(nn.BatchNorm2d):
    def forward(self, x):
        y = x
        y = y.view(x.size(1), -1)
        mu = y.mean(dim=1)
        sigma2 = y.var(dim=1)
        if self.training is not True:
            y = y - self.running_mean.view(-1, 1)
            y = y / (self.running_var.view(-1, 1)**.5 + self.eps)
        else:
            if self.track_running_stats is True:
                with torch.no_grad():
                    self.running_mean = (1-self.momentum)*self.running_mean + self.momentum*mu
                    self.running_var = (1-self.momentum)*self.running_var + self.momentum*sigma2
            y = y - mu.view(-1,1)
            y = y / (sigma2.view(-1,1)**.5 + self.eps)

        y = self.weight.view(-1, 1) * y + self.bias.view(-1, 1)
        return y.view(x.shape)

In [105]:
class LeNet2(nn.Module):
    def __init__(self):
        super(LeNet2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.prelu1 = nn.PReLU()
#        self.bn1 = nn.BatchNorm2d(6)
        self.bn1 = CustomBatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.prelu2 = nn.PReLU()
#        self.bn2 = nn.BatchNorm2d(16)
        self.bn2 = CustomBatchNorm2d(16)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(self.bn1(self.prelu1(self.conv1(x))), (2,2))
        x = F.max_pool2d(self.bn2(self.prelu2(self.conv2(x))), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.1)
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net2 = LeNet2()
print (net2)

LeNet2(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (prelu1): PReLU(num_parameters=1)
  (bn1): CustomBatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (prelu2): PReLU(num_parameters=1)
  (bn2): CustomBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [106]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [107]:
for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        out = net2(x)
        loss = criterion(out, y)
        # оптимизация параметров
        ## первым шагом обнулим градиенты предыдущего шага (важный момент,
        ## без этого результаты теряют корректность)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # подсчет статистики за эпоху        
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))




  0%|          | 0/1875 [00:00<?, ?it/s]


  0%|          | 2/1875 [00:00<01:40, 18.62it/s]


  0%|          | 4/1875 [00:00<01:39, 18.86it/s]


  0%|          | 7/1875 [00:00<01:33, 20.02it/s]


  1%|          | 10/1875 [00:00<01:27, 21.21it/s]


  1%|          | 13/1875 [00:00<01:24, 22.13it/s]


  1%|          | 16/1875 [00:00<01:23, 22.36it/s]


  1%|          | 19/1875 [00:00<01:22, 22.59it/s]


  1%|          | 22/1875 [00:00<01:25, 21.70it/s]


  1%|▏         | 25/1875 [00:01<01:25, 21.57it/s]


  1%|▏         | 28/1875 [00:01<01:31, 20.13it/s]


  2%|▏         | 30/1875 [00:01<01:37, 18.99it/s]


  2%|▏         | 32/1875 [00:01<01:43, 17.89it/s]


  2%|▏         | 34/1875 [00:01<01:47, 17.13it/s]


  2%|▏         | 36/1875 [00:01<02:00, 15.30it/s]


  2%|▏         | 39/1875 [00:01<01:47, 17.13it/s]


  2%|▏         | 42/1875 [00:02<01:48, 16.97it/s]


  2%|▏         | 44/1875 [00:02<01:47, 17.07it/s]


  2%|▏         | 46/1875 [00:02<01:46, 17.16it/s]


  3%|▎         | 48/1

 43%|████▎     | 799/1875 [00:34<00:42, 25.51it/s]


 43%|████▎     | 803/1875 [00:34<00:39, 27.47it/s]


 43%|████▎     | 807/1875 [00:34<00:37, 28.41it/s]


 43%|████▎     | 810/1875 [00:34<00:37, 28.74it/s]


 43%|████▎     | 813/1875 [00:34<00:38, 27.92it/s]


 44%|████▎     | 816/1875 [00:35<00:39, 26.97it/s]


 44%|████▎     | 819/1875 [00:35<00:39, 26.49it/s]


 44%|████▍     | 822/1875 [00:35<00:42, 24.58it/s]


 44%|████▍     | 825/1875 [00:35<00:42, 24.63it/s]


 44%|████▍     | 828/1875 [00:35<00:43, 24.24it/s]


 44%|████▍     | 831/1875 [00:35<00:43, 23.88it/s]


 44%|████▍     | 834/1875 [00:35<00:44, 23.50it/s]


 45%|████▍     | 837/1875 [00:35<00:43, 23.69it/s]


 45%|████▍     | 840/1875 [00:36<00:44, 23.29it/s]


 45%|████▍     | 843/1875 [00:36<00:47, 21.88it/s]


 45%|████▌     | 846/1875 [00:36<00:45, 22.45it/s]


 45%|████▌     | 849/1875 [00:36<00:47, 21.50it/s]


 45%|████▌     | 852/1875 [00:36<00:44, 22.76it/s]


 46%|████▌     | 855/1875 [00:36<00:44, 22.84i

 65%|██████▌   | 1219/1875 [00:51<00:25, 25.90it/s]


 65%|██████▌   | 1222/1875 [00:51<00:24, 26.31it/s]


 65%|██████▌   | 1226/1875 [00:51<00:23, 28.06it/s]


 66%|██████▌   | 1230/1875 [00:51<00:21, 29.35it/s]


 66%|██████▌   | 1234/1875 [00:51<00:20, 30.69it/s]


 66%|██████▌   | 1238/1875 [00:51<00:20, 31.21it/s]


 66%|██████▌   | 1242/1875 [00:52<00:19, 31.74it/s]


 66%|██████▋   | 1246/1875 [00:52<00:20, 31.12it/s]


 67%|██████▋   | 1250/1875 [00:52<00:20, 31.06it/s]


 67%|██████▋   | 1254/1875 [00:52<00:20, 30.51it/s]


 67%|██████▋   | 1258/1875 [00:52<00:20, 30.37it/s]


 67%|██████▋   | 1262/1875 [00:52<00:19, 31.36it/s]


 68%|██████▊   | 1266/1875 [00:52<00:19, 30.97it/s]


 68%|██████▊   | 1270/1875 [00:52<00:19, 31.47it/s]


 68%|██████▊   | 1274/1875 [00:53<00:18, 32.21it/s]


 68%|██████▊   | 1278/1875 [00:53<00:18, 32.17it/s]


 68%|██████▊   | 1282/1875 [00:53<00:19, 30.55it/s]


 69%|██████▊   | 1286/1875 [00:53<00:18, 31.34it/s]


 69%|██████▉   | 1290/1875 [

Epoch: 0; Accuracy train: 10.327%





  0%|          | 6/1875 [00:00<01:05, 28.65it/s]


  0%|          | 9/1875 [00:00<01:04, 28.85it/s]


  1%|          | 12/1875 [00:00<01:04, 28.67it/s]


  1%|          | 16/1875 [00:00<01:00, 30.50it/s]


  1%|          | 20/1875 [00:00<01:00, 30.85it/s]


  1%|          | 23/1875 [00:00<01:00, 30.57it/s]


  1%|▏         | 26/1875 [00:00<01:00, 30.33it/s]


  2%|▏         | 29/1875 [00:00<01:01, 30.04it/s]


  2%|▏         | 32/1875 [00:01<01:02, 29.58it/s]


  2%|▏         | 36/1875 [00:01<01:00, 30.57it/s]


  2%|▏         | 40/1875 [00:01<00:59, 30.85it/s]


  2%|▏         | 44/1875 [00:01<00:59, 30.96it/s]


  3%|▎         | 48/1875 [00:01<00:57, 31.78it/s]


  3%|▎         | 52/1875 [00:01<00:57, 31.52it/s]


  3%|▎         | 56/1875 [00:01<00:58, 31.13it/s]


  3%|▎         | 60/1875 [00:01<00:59, 30.73it/s]


  3%|▎         | 64/1875 [00:02<00:59, 30.68it/s]


  4%|▎         | 68/1875 [00:02<00:59, 30.34it/s]


  4%|▍         | 72/1875 [00:02<00:57, 31.14it/s]


  4%|▍     

...

Epoch: 8; Accuracy train: 10.368%





  0%|          | 8/1875 [00:00<00:54, 34.21it/s]


  1%|          | 12/1875 [00:00<00:52, 35.54it/s]


  1%|          | 16/1875 [00:00<00:53, 34.99it/s]


  1%|          | 20/1875 [00:00<00:51, 35.99it/s]


  1%|▏         | 24/1875 [00:00<00:49, 37.09it/s]


  1%|▏         | 28/1875 [00:00<00:49, 37.47it/s]


  2%|▏         | 32/1875 [00:00<00:50, 36.67it/s]


  2%|▏         | 36/1875 [00:00<00:51, 35.90it/s]


  2%|▏         | 40/1875 [00:01<00:51, 35.71it/s]


  2%|▏         | 44/1875 [00:01<00:50, 36.60it/s]


  3%|▎         | 48/1875 [00:01<00:48, 37.44it/s]


  3%|▎         | 52/1875 [00:01<00:48, 37.87it/s]


  3%|▎         | 56/1875 [00:01<00:47, 38.00it/s]


  3%|▎         | 60/1875 [00:01<00:48, 37.78it/s]


  3%|▎         | 64/1875 [00:01<00:47, 37.74it/s]


  4%|▎         | 68/1875 [00:01<00:50, 35.83it/s]


  4%|▍         | 72/1875 [00:01<00:50, 35.65it/s]


  4%|▍         | 76/1875 [00:02<00:50, 35.56it/s]


  4%|▍         | 80/1875 [00:02<00:48, 36.64it/s]


  4%|▍    

 45%|████▌     | 853/1875 [00:30<00:39, 25.83it/s]


 46%|████▌     | 856/1875 [00:30<00:39, 25.82it/s]


 46%|████▌     | 859/1875 [00:30<00:38, 26.67it/s]


 46%|████▌     | 862/1875 [00:30<00:37, 26.99it/s]


 46%|████▌     | 865/1875 [00:30<00:37, 27.21it/s]


 46%|████▋     | 868/1875 [00:30<00:37, 27.09it/s]


 46%|████▋     | 871/1875 [00:30<00:37, 26.78it/s]


 47%|████▋     | 874/1875 [00:30<00:37, 26.83it/s]


 47%|████▋     | 877/1875 [00:31<00:38, 26.15it/s]


 47%|████▋     | 880/1875 [00:31<00:38, 25.70it/s]


 47%|████▋     | 883/1875 [00:31<00:37, 26.17it/s]


 47%|████▋     | 886/1875 [00:31<00:37, 26.50it/s]


 47%|████▋     | 889/1875 [00:31<00:37, 26.45it/s]


 48%|████▊     | 892/1875 [00:31<00:36, 26.86it/s]


 48%|████▊     | 895/1875 [00:31<00:36, 27.21it/s]


 48%|████▊     | 898/1875 [00:31<00:37, 26.16it/s]


 48%|████▊     | 901/1875 [00:32<00:35, 27.08it/s]


 48%|████▊     | 904/1875 [00:32<00:36, 26.64it/s]


 48%|████▊     | 907/1875 [00:32<00:36, 26.64i

 98%|█████████▊| 1834/1875 [00:59<00:01, 36.76it/s]


 98%|█████████▊| 1838/1875 [00:59<00:00, 37.35it/s]


 98%|█████████▊| 1842/1875 [00:59<00:00, 38.03it/s]


 98%|█████████▊| 1846/1875 [00:59<00:00, 37.48it/s]


 99%|█████████▊| 1850/1875 [00:59<00:00, 37.90it/s]


 99%|█████████▉| 1854/1875 [00:59<00:00, 38.25it/s]


 99%|█████████▉| 1858/1875 [00:59<00:00, 37.05it/s]


 99%|█████████▉| 1862/1875 [01:00<00:00, 36.56it/s]


100%|█████████▉| 1866/1875 [01:00<00:00, 36.21it/s]


100%|█████████▉| 1870/1875 [01:00<00:00, 36.99it/s]


100%|█████████▉| 1874/1875 [01:00<00:00, 37.82it/s]


100%|██████████| 1875/1875 [01:00<00:00, 31.07it/s]

Epoch: 9; Accuracy train: 10.248%


In [ ]:
av_loss = 0.
correct = 0.
for x, y in tqdm.tqdm(test_loader):
    # рассчитываем функцию потерь
    out = net2(x)
    loss = criterion(out, y)
    # подсчет статистики за эпоху        
    pred = out.max(1, keepdim=True)[1]
    correct += pred.eq(y.view_as(pred)).sum().item()
    av_loss += loss.item()
print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(test_loader.dataset) * 100))

#  RMSprop

In [85]:
from torch.optim.optimizer import Optimizer, required
import copy

class MyRMSProp(Optimizer):
    
    def __init__(self, params, lr=1e-2, alpha=0.9, eps=1e-8):
        defaults = dict(lr=lr, alpha=alpha, eps=eps)
        super(MyRMSProp, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(MyRMSProp, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            alpha = group['alpha']
            lr = group['lr']
            eps = group['eps']
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                state = self.state[p]
                if len(state) == 0:
                    state['grad_squared'] = torch.zeros_like(p.data)

                grad_squared = state['grad_squared']

                grad_squared.mul_(alpha)
                grad_squared.addcmul_(1-alpha, d_p, d_p)

                grad_avg = grad_squared.sqrt().add_(eps)

                p.data.addcdiv_(-lr, d_p, grad_avg)

        return loss

In [86]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.prelu1 = nn.PReLU()
        self.bn1 = nn.BatchNorm2d(6)
#        self.bn1 = CustomBatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.prelu2 = nn.PReLU()
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(self.bn1(self.prelu1(self.conv1(x))), (2,2))
        x = F.max_pool2d(self.bn2(self.prelu2(self.conv2(x))), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.1)
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [87]:
net_rmsprop = LeNet()
print (net_rmsprop)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (prelu1): PReLU(num_parameters=1)
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (prelu2): PReLU(num_parameters=1)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [88]:
optimizer = MyRMSProp(net_rmsprop.parameters())
criterion = nn.CrossEntropyLoss()

In [89]:
NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        out = net_rmsprop(x)
        loss = criterion(out, y)
        # оптимизация параметров
        ## первым шагом обнулим градиенты предыдущего шага (важный момент,
        ## без этого результаты теряют корректность)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # подсчет статистики за эпоху        
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))






  0%|          | 0/1875 [00:00<?, ?it/s]




  0%|          | 4/1875 [00:00<00:55, 33.90it/s]




  0%|          | 7/1875 [00:00<00:57, 32.55it/s]




  0%|          | 9/1875 [00:00<01:08, 27.29it/s]




  1%|          | 12/1875 [00:00<01:07, 27.46it/s]




  1%|          | 15/1875 [00:00<01:07, 27.41it/s]




  1%|          | 18/1875 [00:00<01:06, 28.07it/s]




  1%|          | 21/1875 [00:00<01:08, 26.89it/s]




  1%|▏         | 24/1875 [00:00<01:07, 27.29it/s]




  1%|▏         | 27/1875 [00:00<01:08, 26.84it/s]




  2%|▏         | 30/1875 [00:01<01:09, 26.59it/s]




  2%|▏         | 33/1875 [00:01<01:08, 27.04it/s]




  2%|▏         | 36/1875 [00:01<01:07, 27.06it/s]




  2%|▏         | 39/1875 [00:01<01:10, 26.16it/s]




  2%|▏         | 43/1875 [00:01<01:06, 27.41it/s]




  2%|▏         | 46/1875 [00:01<01:10, 25.78it/s]




  3%|▎         | 49/1875 [00:01<01:11, 25.50it/s]




  3%|▎         | 52/1875 [00:01<01:08, 26.51it/s]




  3%|▎         | 55/1875 [00:02<01:

 20%|█▉        | 370/1875 [00:14<01:01, 24.62it/s]




 20%|█▉        | 373/1875 [00:14<01:01, 24.29it/s]




 20%|██        | 377/1875 [00:14<00:55, 26.92it/s]




 20%|██        | 381/1875 [00:14<00:50, 29.33it/s]




 21%|██        | 385/1875 [00:14<00:47, 31.49it/s]




 21%|██        | 390/1875 [00:14<00:43, 34.06it/s]




 21%|██        | 394/1875 [00:14<00:42, 34.60it/s]




 21%|██▏       | 399/1875 [00:14<00:40, 36.36it/s]




 21%|██▏       | 403/1875 [00:15<00:39, 37.27it/s]




 22%|██▏       | 407/1875 [00:15<00:40, 35.97it/s]




 22%|██▏       | 411/1875 [00:15<00:45, 32.41it/s]




 22%|██▏       | 415/1875 [00:15<00:43, 33.33it/s]




 22%|██▏       | 419/1875 [00:15<00:45, 32.03it/s]




 23%|██▎       | 423/1875 [00:15<00:47, 30.78it/s]




 23%|██▎       | 427/1875 [00:15<00:48, 29.74it/s]




 23%|██▎       | 431/1875 [00:15<00:48, 29.71it/s]




 23%|██▎       | 435/1875 [00:16<00:48, 29.44it/s]




 23%|██▎       | 438/1875 [00:16<00:51, 28.07it/s]




 24%|██▎  

 42%|████▏     | 788/1875 [00:29<00:38, 28.24it/s]




 42%|████▏     | 791/1875 [00:29<00:38, 28.36it/s]




 42%|████▏     | 795/1875 [00:29<00:35, 30.08it/s]




 43%|████▎     | 799/1875 [00:29<00:35, 30.52it/s]




 43%|████▎     | 803/1875 [00:29<00:34, 31.10it/s]




 43%|████▎     | 807/1875 [00:29<00:33, 32.26it/s]




 43%|████▎     | 811/1875 [00:29<00:32, 32.61it/s]




 43%|████▎     | 815/1875 [00:29<00:30, 34.35it/s]




 44%|████▎     | 819/1875 [00:29<00:29, 35.50it/s]




 44%|████▍     | 823/1875 [00:30<00:30, 34.86it/s]




 44%|████▍     | 827/1875 [00:30<00:29, 35.25it/s]




 44%|████▍     | 831/1875 [00:30<00:32, 32.57it/s]




 45%|████▍     | 835/1875 [00:30<00:31, 33.14it/s]




 45%|████▍     | 839/1875 [00:30<00:30, 34.07it/s]




 45%|████▍     | 843/1875 [00:30<00:29, 35.13it/s]




 45%|████▌     | 847/1875 [00:30<00:28, 36.11it/s]




 45%|████▌     | 851/1875 [00:30<00:28, 36.11it/s]




 46%|████▌     | 855/1875 [00:30<00:29, 35.12it/s]




 46%|████▌

 64%|██████▍   | 1199/1875 [00:43<00:24, 27.19it/s]




 64%|██████▍   | 1202/1875 [00:43<00:24, 27.39it/s]




 64%|██████▍   | 1205/1875 [00:43<00:24, 26.90it/s]




 64%|██████▍   | 1208/1875 [00:43<00:24, 27.69it/s]




 65%|██████▍   | 1212/1875 [00:43<00:22, 28.97it/s]




 65%|██████▍   | 1215/1875 [00:43<00:23, 28.30it/s]




 65%|██████▍   | 1218/1875 [00:43<00:23, 27.95it/s]




 65%|██████▌   | 1221/1875 [00:43<00:23, 27.49it/s]




 65%|██████▌   | 1224/1875 [00:43<00:23, 27.88it/s]




 65%|██████▌   | 1227/1875 [00:44<00:25, 25.79it/s]




 66%|██████▌   | 1230/1875 [00:44<00:25, 24.94it/s]




 66%|██████▌   | 1233/1875 [00:44<00:26, 23.94it/s]




 66%|██████▌   | 1236/1875 [00:44<00:26, 24.07it/s]




 66%|██████▌   | 1239/1875 [00:44<00:25, 25.32it/s]




 66%|██████▌   | 1242/1875 [00:44<00:24, 25.59it/s]




 66%|██████▋   | 1245/1875 [00:44<00:24, 25.99it/s]




 67%|██████▋   | 1248/1875 [00:44<00:23, 26.87it/s]




 67%|██████▋   | 1252/1875 [00:45<00:22, 28.23it

 83%|████████▎ | 1556/1875 [00:56<00:12, 25.20it/s]




 83%|████████▎ | 1559/1875 [00:56<00:13, 23.67it/s]




 83%|████████▎ | 1562/1875 [00:56<00:13, 23.48it/s]




 83%|████████▎ | 1565/1875 [00:57<00:13, 22.70it/s]




 84%|████████▎ | 1568/1875 [00:57<00:15, 20.39it/s]




 84%|████████▍ | 1571/1875 [00:57<00:13, 22.27it/s]




 84%|████████▍ | 1574/1875 [00:57<00:13, 22.14it/s]




 84%|████████▍ | 1577/1875 [00:57<00:13, 21.79it/s]




 84%|████████▍ | 1580/1875 [00:57<00:13, 21.40it/s]




 84%|████████▍ | 1583/1875 [00:57<00:12, 23.05it/s]




 85%|████████▍ | 1586/1875 [00:58<00:12, 23.08it/s]




 85%|████████▍ | 1589/1875 [00:58<00:12, 23.41it/s]




 85%|████████▍ | 1592/1875 [00:58<00:13, 21.17it/s]




 85%|████████▌ | 1595/1875 [00:58<00:12, 22.17it/s]




 85%|████████▌ | 1598/1875 [00:58<00:12, 23.03it/s]




 85%|████████▌ | 1602/1875 [00:58<00:10, 25.23it/s]




 86%|████████▌ | 1605/1875 [00:58<00:10, 26.20it/s]




 86%|████████▌ | 1608/1875 [00:58<00:09, 26.75it

...

 23%|██▎       | 436/1875 [00:15<00:43, 33.34it/s]




 23%|██▎       | 440/1875 [00:15<00:42, 33.56it/s]




 24%|██▎       | 444/1875 [00:15<00:44, 32.10it/s]




 24%|██▍       | 448/1875 [00:16<00:43, 32.43it/s]




 24%|██▍       | 452/1875 [00:16<00:42, 33.09it/s]




 24%|██▍       | 456/1875 [00:16<00:45, 31.33it/s]




 25%|██▍       | 460/1875 [00:16<00:44, 31.84it/s]




 25%|██▍       | 464/1875 [00:16<00:49, 28.28it/s]




 25%|██▍       | 467/1875 [00:16<00:51, 27.31it/s]




 25%|██▌       | 471/1875 [00:16<00:48, 28.71it/s]




 25%|██▌       | 475/1875 [00:16<00:46, 30.09it/s]




 26%|██▌       | 479/1875 [00:17<00:44, 31.27it/s]




 26%|██▌       | 483/1875 [00:17<00:43, 32.04it/s]




 26%|██▌       | 487/1875 [00:17<00:45, 30.58it/s]




 26%|██▌       | 491/1875 [00:17<00:44, 31.42it/s]




 26%|██▋       | 495/1875 [00:17<00:43, 31.39it/s]




 27%|██▋       | 499/1875 [00:17<00:42, 32.04it/s]




 27%|██▋       | 503/1875 [00:17<00:41, 32.73it/s]




 27%|██▋  

 45%|████▍     | 836/1875 [00:31<01:09, 15.00it/s]




 45%|████▍     | 838/1875 [00:31<01:12, 14.29it/s]




 45%|████▍     | 840/1875 [00:31<01:18, 13.20it/s]




 45%|████▍     | 842/1875 [00:31<01:24, 12.15it/s]




 45%|████▌     | 844/1875 [00:31<01:20, 12.76it/s]




 45%|████▌     | 847/1875 [00:31<01:12, 14.22it/s]




 45%|████▌     | 849/1875 [00:32<01:09, 14.73it/s]




 45%|████▌     | 851/1875 [00:32<01:04, 15.77it/s]




 45%|████▌     | 853/1875 [00:32<01:01, 16.74it/s]




 46%|████▌     | 855/1875 [00:32<00:59, 17.21it/s]




 46%|████▌     | 857/1875 [00:32<00:56, 17.88it/s]




 46%|████▌     | 860/1875 [00:32<00:51, 19.59it/s]




 46%|████▌     | 863/1875 [00:32<00:53, 19.09it/s]




 46%|████▌     | 865/1875 [00:32<00:53, 18.88it/s]




 46%|████▌     | 867/1875 [00:33<00:55, 18.17it/s]




 46%|████▋     | 869/1875 [00:33<00:56, 17.94it/s]




 46%|████▋     | 871/1875 [00:33<01:02, 16.17it/s]




 47%|████▋     | 873/1875 [00:33<01:06, 15.13it/s]




 47%|████▋

 62%|██████▏   | 1164/1875 [00:46<00:19, 36.82it/s]




 62%|██████▏   | 1168/1875 [00:46<00:18, 37.55it/s]




 63%|██████▎   | 1172/1875 [00:46<00:18, 37.37it/s]




 63%|██████▎   | 1177/1875 [00:46<00:17, 38.82it/s]




 63%|██████▎   | 1182/1875 [00:46<00:17, 39.95it/s]




 63%|██████▎   | 1187/1875 [00:46<00:16, 40.53it/s]




 64%|██████▎   | 1192/1875 [00:46<00:16, 41.26it/s]




 64%|██████▍   | 1197/1875 [00:46<00:16, 41.73it/s]




 64%|██████▍   | 1202/1875 [00:47<00:16, 41.87it/s]




 64%|██████▍   | 1207/1875 [00:47<00:17, 38.44it/s]




 65%|██████▍   | 1211/1875 [00:47<00:19, 33.21it/s]




 65%|██████▍   | 1216/1875 [00:47<00:18, 35.25it/s]




 65%|██████▌   | 1221/1875 [00:47<00:17, 36.83it/s]




 65%|██████▌   | 1225/1875 [00:47<00:21, 29.91it/s]




 66%|██████▌   | 1229/1875 [00:47<00:20, 32.25it/s]




 66%|██████▌   | 1234/1875 [00:47<00:18, 34.79it/s]




 66%|██████▌   | 1239/1875 [00:48<00:17, 36.55it/s]




 66%|██████▋   | 1244/1875 [00:48<00:16, 37.20it

 88%|████████▊ | 1644/1875 [01:00<00:09, 24.17it/s]




 88%|████████▊ | 1647/1875 [01:00<00:09, 22.85it/s]




 88%|████████▊ | 1650/1875 [01:01<00:10, 21.80it/s]




 88%|████████▊ | 1654/1875 [01:01<00:09, 23.90it/s]




 88%|████████▊ | 1658/1875 [01:01<00:08, 27.03it/s]




 89%|████████▊ | 1663/1875 [01:01<00:06, 30.41it/s]




 89%|████████▉ | 1668/1875 [01:01<00:06, 33.23it/s]




 89%|████████▉ | 1673/1875 [01:01<00:05, 35.46it/s]




 89%|████████▉ | 1678/1875 [01:01<00:05, 37.22it/s]




 90%|████████▉ | 1683/1875 [01:01<00:04, 38.66it/s]




 90%|█████████ | 1688/1875 [01:02<00:04, 39.08it/s]




 90%|█████████ | 1693/1875 [01:02<00:04, 39.02it/s]




 91%|█████████ | 1697/1875 [01:02<00:04, 37.19it/s]




 91%|█████████ | 1702/1875 [01:02<00:04, 38.66it/s]




 91%|█████████ | 1707/1875 [01:02<00:04, 39.72it/s]




 91%|█████████▏| 1712/1875 [01:02<00:04, 39.52it/s]




 92%|█████████▏| 1717/1875 [01:02<00:03, 40.16it/s]




 92%|█████████▏| 1722/1875 [01:02<00:04, 35.98it

Epoch: 9; Accuracy train: 96.022%


# SGD vs. RMSProp

In [90]:
net_sgd = LeNet()
print (net_sgd)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (prelu1): PReLU(num_parameters=1)
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (prelu2): PReLU(num_parameters=1)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [91]:
optimizer = optim.SGD(net_sgd.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [92]:
NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    # trainning
    av_loss = 0.
    correct = 0.
    for x, y in tqdm.tqdm(train_loader):
        # рассчитываем функцию потерь
        out = net_sgd(x)
        loss = criterion(out, y)
        # оптимизация параметров
        ## первым шагом обнулим градиенты предыдущего шага (важный момент,
        ## без этого результаты теряют корректность)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # подсчет статистики за эпоху        
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        av_loss += loss.item()
    print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(train_loader.dataset) * 100))






  0%|          | 0/1875 [00:00<?, ?it/s]




  0%|          | 2/1875 [00:00<01:40, 18.72it/s]




  0%|          | 5/1875 [00:00<01:28, 21.06it/s]




  0%|          | 9/1875 [00:00<01:17, 23.99it/s]




  1%|          | 13/1875 [00:00<01:10, 26.42it/s]




  1%|          | 17/1875 [00:00<01:05, 28.58it/s]




  1%|          | 20/1875 [00:00<01:03, 28.99it/s]




  1%|▏         | 24/1875 [00:00<01:00, 30.43it/s]




  1%|▏         | 28/1875 [00:00<00:56, 32.52it/s]




  2%|▏         | 32/1875 [00:00<00:53, 34.41it/s]




  2%|▏         | 36/1875 [00:01<00:57, 32.11it/s]




  2%|▏         | 40/1875 [00:01<00:55, 32.82it/s]




  2%|▏         | 44/1875 [00:01<00:54, 33.68it/s]




  3%|▎         | 48/1875 [00:01<00:53, 34.28it/s]




  3%|▎         | 52/1875 [00:01<00:51, 35.11it/s]




  3%|▎         | 56/1875 [00:01<00:51, 35.37it/s]




  3%|▎         | 60/1875 [00:01<00:50, 35.60it/s]




  3%|▎         | 64/1875 [00:01<00:52, 34.19it/s]




  4%|▎         | 68/1875 [00:02<00:

 23%|██▎       | 423/1875 [00:14<00:51, 28.07it/s]




 23%|██▎       | 426/1875 [00:14<00:51, 28.18it/s]




 23%|██▎       | 429/1875 [00:15<00:55, 26.01it/s]




 23%|██▎       | 433/1875 [00:15<00:51, 27.90it/s]




 23%|██▎       | 436/1875 [00:15<00:57, 24.86it/s]




 23%|██▎       | 439/1875 [00:15<01:01, 23.47it/s]




 24%|██▎       | 442/1875 [00:15<00:59, 23.91it/s]




 24%|██▎       | 445/1875 [00:15<00:58, 24.27it/s]




 24%|██▍       | 449/1875 [00:15<00:54, 26.33it/s]




 24%|██▍       | 452/1875 [00:15<00:53, 26.54it/s]




 24%|██▍       | 455/1875 [00:16<00:55, 25.67it/s]




 24%|██▍       | 459/1875 [00:16<00:50, 27.93it/s]




 25%|██▍       | 462/1875 [00:16<01:01, 23.05it/s]




 25%|██▍       | 465/1875 [00:16<01:01, 23.02it/s]




 25%|██▍       | 468/1875 [00:16<01:03, 21.99it/s]




 25%|██▌       | 471/1875 [00:16<01:03, 22.12it/s]




 25%|██▌       | 475/1875 [00:16<00:56, 24.77it/s]




 25%|██▌       | 478/1875 [00:16<00:53, 26.13it/s]




 26%|██▌  

 47%|████▋     | 875/1875 [00:29<00:26, 37.91it/s]




 47%|████▋     | 879/1875 [00:29<00:26, 37.44it/s]




 47%|████▋     | 883/1875 [00:29<00:26, 37.16it/s]




 47%|████▋     | 887/1875 [00:30<00:28, 35.22it/s]




 48%|████▊     | 891/1875 [00:30<00:27, 35.96it/s]




 48%|████▊     | 896/1875 [00:30<00:26, 37.32it/s]




 48%|████▊     | 900/1875 [00:30<00:26, 37.38it/s]




 48%|████▊     | 904/1875 [00:30<00:26, 36.92it/s]




 48%|████▊     | 908/1875 [00:30<00:26, 37.18it/s]




 49%|████▊     | 912/1875 [00:30<00:25, 37.28it/s]




 49%|████▉     | 916/1875 [00:30<00:25, 37.21it/s]




 49%|████▉     | 920/1875 [00:30<00:25, 37.84it/s]




 49%|████▉     | 924/1875 [00:31<00:26, 35.89it/s]




 49%|████▉     | 928/1875 [00:31<00:27, 34.98it/s]




 50%|████▉     | 932/1875 [00:31<00:26, 35.81it/s]




 50%|████▉     | 936/1875 [00:31<00:26, 35.86it/s]




 50%|█████     | 940/1875 [00:31<00:25, 36.17it/s]




 50%|█████     | 944/1875 [00:31<00:25, 36.03it/s]




 51%|█████

 73%|███████▎  | 1371/1875 [00:43<00:14, 33.69it/s]




 73%|███████▎  | 1375/1875 [00:44<00:16, 30.84it/s]




 74%|███████▎  | 1379/1875 [00:44<00:16, 29.70it/s]




 74%|███████▍  | 1383/1875 [00:44<00:18, 27.10it/s]




 74%|███████▍  | 1387/1875 [00:44<00:17, 27.33it/s]




 74%|███████▍  | 1390/1875 [00:44<00:17, 27.14it/s]




 74%|███████▍  | 1394/1875 [00:44<00:17, 28.09it/s]




 75%|███████▍  | 1397/1875 [00:44<00:17, 27.93it/s]




 75%|███████▍  | 1400/1875 [00:45<00:20, 23.04it/s]




 75%|███████▍  | 1403/1875 [00:45<00:23, 20.08it/s]




 75%|███████▍  | 1406/1875 [00:45<00:25, 18.76it/s]




 75%|███████▌  | 1409/1875 [00:45<00:22, 20.55it/s]




 75%|███████▌  | 1412/1875 [00:45<00:22, 20.74it/s]




 76%|███████▌  | 1416/1875 [00:45<00:19, 23.21it/s]




 76%|███████▌  | 1419/1875 [00:46<00:19, 22.98it/s]




 76%|███████▌  | 1422/1875 [00:46<00:20, 22.11it/s]




 76%|███████▌  | 1425/1875 [00:46<00:20, 21.99it/s]




 76%|███████▌  | 1428/1875 [00:46<00:21, 20.47it

 97%|█████████▋| 1815/1875 [00:58<00:01, 30.53it/s]




 97%|█████████▋| 1819/1875 [00:58<00:02, 25.40it/s]




 97%|█████████▋| 1824/1875 [00:58<00:01, 29.19it/s]




 98%|█████████▊| 1829/1875 [00:58<00:01, 32.00it/s]




 98%|█████████▊| 1833/1875 [00:58<00:01, 27.15it/s]




 98%|█████████▊| 1837/1875 [00:58<00:01, 30.01it/s]




 98%|█████████▊| 1841/1875 [00:59<00:01, 31.34it/s]




 98%|█████████▊| 1845/1875 [00:59<00:00, 32.67it/s]




 99%|█████████▊| 1849/1875 [00:59<00:00, 29.78it/s]




 99%|█████████▉| 1854/1875 [00:59<00:00, 33.25it/s]




 99%|█████████▉| 1859/1875 [00:59<00:00, 36.71it/s]




 99%|█████████▉| 1864/1875 [00:59<00:00, 39.69it/s]




100%|█████████▉| 1869/1875 [00:59<00:00, 35.12it/s]




100%|█████████▉| 1873/1875 [01:00<00:00, 32.41it/s]




100%|██████████| 1875/1875 [01:00<00:00, 31.21it/s]




  0%|          | 0/1875 [00:00<?, ?it/s]




  0%|          | 4/1875 [00:00<00:48, 38.32it/s]

...






  0%|          | 6/1875 [00:00<01:10, 26.53it/s]




  1%|          | 10/1875 [00:00<01:03, 29.21it/s]




  1%|          | 14/1875 [00:00<00:58, 31.70it/s]




  1%|          | 18/1875 [00:00<00:55, 33.75it/s]




  1%|          | 22/1875 [00:00<00:57, 32.14it/s]




  1%|▏         | 26/1875 [00:00<00:55, 33.60it/s]




  2%|▏         | 30/1875 [00:00<00:53, 34.78it/s]




  2%|▏         | 34/1875 [00:00<00:52, 35.31it/s]




  2%|▏         | 38/1875 [00:01<00:56, 32.68it/s]




  2%|▏         | 42/1875 [00:01<00:58, 31.60it/s]




  2%|▏         | 46/1875 [00:01<00:56, 32.28it/s]




  3%|▎         | 50/1875 [00:01<00:59, 30.64it/s]




  3%|▎         | 54/1875 [00:01<01:04, 28.37it/s]




  3%|▎         | 57/1875 [00:01<01:09, 26.09it/s]




  3%|▎         | 61/1875 [00:01<01:02, 28.94it/s]




  4%|▎         | 66/1875 [00:02<00:55, 32.89it/s]




  4%|▍         | 71/1875 [00:02<00:49, 36.44it/s]




  4%|▍         | 75/1875 [00:02<00:51, 35.08it/s]




  4%|▍         | 79/1875

 28%|██▊       | 519/1875 [00:14<00:36, 37.51it/s]




 28%|██▊       | 523/1875 [00:14<00:36, 37.36it/s]




 28%|██▊       | 528/1875 [00:14<00:34, 38.60it/s]




 28%|██▊       | 532/1875 [00:14<00:34, 38.43it/s]




 29%|██▊       | 536/1875 [00:14<00:34, 38.61it/s]




 29%|██▉       | 540/1875 [00:14<00:35, 37.36it/s]




 29%|██▉       | 544/1875 [00:14<00:35, 37.18it/s]




 29%|██▉       | 549/1875 [00:14<00:34, 38.37it/s]




 30%|██▉       | 554/1875 [00:14<00:33, 39.13it/s]




 30%|██▉       | 559/1875 [00:15<00:32, 40.04it/s]




 30%|███       | 564/1875 [00:15<00:32, 40.84it/s]




 30%|███       | 569/1875 [00:15<00:31, 40.90it/s]




 31%|███       | 574/1875 [00:15<00:31, 41.13it/s]




 31%|███       | 579/1875 [00:15<00:31, 41.36it/s]




 31%|███       | 584/1875 [00:15<00:31, 41.26it/s]




 31%|███▏      | 589/1875 [00:15<00:30, 41.67it/s]




 32%|███▏      | 594/1875 [00:15<00:30, 41.60it/s]




 32%|███▏      | 599/1875 [00:15<00:30, 41.87it/s]




 32%|███▏ 

 56%|█████▋    | 1057/1875 [00:27<00:22, 36.83it/s]




 57%|█████▋    | 1061/1875 [00:27<00:23, 33.97it/s]




 57%|█████▋    | 1065/1875 [00:27<00:24, 33.51it/s]




 57%|█████▋    | 1070/1875 [00:28<00:22, 35.93it/s]




 57%|█████▋    | 1076/1875 [00:28<00:20, 39.39it/s]




 58%|█████▊    | 1081/1875 [00:28<00:18, 41.90it/s]




 58%|█████▊    | 1086/1875 [00:28<00:18, 43.71it/s]




 58%|█████▊    | 1091/1875 [00:28<00:17, 45.18it/s]




 58%|█████▊    | 1096/1875 [00:28<00:17, 45.82it/s]




 59%|█████▊    | 1101/1875 [00:28<00:16, 46.84it/s]




 59%|█████▉    | 1106/1875 [00:28<00:16, 47.35it/s]




 59%|█████▉    | 1111/1875 [00:28<00:15, 48.03it/s]




 60%|█████▉    | 1116/1875 [00:29<00:15, 48.46it/s]




 60%|█████▉    | 1121/1875 [00:29<00:15, 48.61it/s]




 60%|██████    | 1126/1875 [00:29<00:17, 42.92it/s]




 60%|██████    | 1131/1875 [00:29<00:17, 42.03it/s]




 61%|██████    | 1136/1875 [00:29<00:18, 39.82it/s]




 61%|██████    | 1141/1875 [00:29<00:17, 41.84it

 87%|████████▋ | 1624/1875 [00:40<00:06, 38.39it/s]




 87%|████████▋ | 1628/1875 [00:41<00:06, 36.05it/s]




 87%|████████▋ | 1632/1875 [00:41<00:06, 34.89it/s]




 87%|████████▋ | 1636/1875 [00:41<00:06, 35.04it/s]




 87%|████████▋ | 1640/1875 [00:41<00:06, 34.11it/s]




 88%|████████▊ | 1645/1875 [00:41<00:06, 36.75it/s]




 88%|████████▊ | 1650/1875 [00:41<00:05, 39.76it/s]




 88%|████████▊ | 1655/1875 [00:41<00:05, 42.15it/s]




 89%|████████▊ | 1660/1875 [00:41<00:05, 41.24it/s]




 89%|████████▉ | 1665/1875 [00:42<00:05, 40.59it/s]




 89%|████████▉ | 1670/1875 [00:42<00:05, 38.83it/s]




 89%|████████▉ | 1674/1875 [00:42<00:05, 36.21it/s]




 89%|████████▉ | 1678/1875 [00:42<00:05, 36.66it/s]




 90%|████████▉ | 1682/1875 [00:42<00:05, 35.49it/s]




 90%|████████▉ | 1686/1875 [00:42<00:05, 35.54it/s]




 90%|█████████ | 1690/1875 [00:42<00:05, 34.15it/s]




 90%|█████████ | 1694/1875 [00:42<00:05, 35.55it/s]




 91%|█████████ | 1698/1875 [00:42<00:05, 35.24it

Epoch: 9; Accuracy train: 99.253%


# RMSProp Accuracy train

In [93]:
av_loss = 0.
correct = 0.
for x, y in tqdm.tqdm(test_loader):
    # рассчитываем функцию потерь
    out = net_rmsprop(x)
    loss = criterion(out, y)
    # подсчет статистики за эпоху        
    pred = out.max(1, keepdim=True)[1]
    correct += pred.eq(y.view_as(pred)).sum().item()
    av_loss += loss.item()
print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(test_loader.dataset) * 100))






  0%|          | 0/313 [00:00<?, ?it/s]




  3%|▎         | 9/313 [00:00<00:03, 84.60it/s]




  7%|▋         | 21/313 [00:00<00:03, 91.75it/s]




 10%|█         | 32/313 [00:00<00:02, 96.33it/s]




 14%|█▍        | 45/313 [00:00<00:02, 103.05it/s]




 18%|█▊        | 56/313 [00:00<00:02, 104.13it/s]




 21%|██▏       | 67/313 [00:00<00:02, 105.17it/s]




 25%|██▍       | 78/313 [00:00<00:02, 105.24it/s]




 28%|██▊       | 88/313 [00:00<00:02, 102.12it/s]




 31%|███▏      | 98/313 [00:00<00:02, 96.72it/s] 




 35%|███▍      | 108/313 [00:01<00:02, 96.69it/s]




 38%|███▊      | 118/313 [00:01<00:02, 92.53it/s]




 41%|████      | 129/313 [00:01<00:01, 96.34it/s]




 44%|████▍     | 139/313 [00:01<00:01, 97.29it/s]




 48%|████▊     | 150/313 [00:01<00:01, 100.29it/s]




 51%|█████▏    | 161/313 [00:01<00:01, 98.97it/s] 




 55%|█████▍    | 172/313 [00:01<00:01, 101.27it/s]




 58%|█████▊    | 183/313 [00:01<00:01, 103.13it/s]




 62%|██████▏   | 194/313 [00:01<0

Epoch: 9; Accuracy train: 96.440%


# SGD Accuracy train

In [94]:
av_loss = 0.
correct = 0.
for x, y in tqdm.tqdm(test_loader):
    # рассчитываем функцию потерь
    out = net_sgd(x)
    loss = criterion(out, y)
    # подсчет статистики за эпоху        
    pred = out.max(1, keepdim=True)[1]
    correct += pred.eq(y.view_as(pred)).sum().item()
    av_loss += loss.item()
print('Epoch: {}; Accuracy train: {:.3f}%'.format(epoch, correct / len(test_loader.dataset) * 100))






  0%|          | 0/313 [00:00<?, ?it/s]




  3%|▎         | 8/313 [00:00<00:04, 72.53it/s]




  5%|▌         | 16/313 [00:00<00:04, 73.54it/s]




  9%|▊         | 27/313 [00:00<00:03, 80.23it/s]




 12%|█▏        | 37/313 [00:00<00:03, 84.14it/s]




 15%|█▌        | 47/313 [00:00<00:03, 88.27it/s]




 18%|█▊        | 57/313 [00:00<00:02, 90.64it/s]




 21%|██▏       | 67/313 [00:00<00:02, 92.07it/s]




 25%|██▍       | 78/313 [00:00<00:02, 96.30it/s]




 28%|██▊       | 89/313 [00:00<00:02, 99.77it/s]




 32%|███▏      | 101/313 [00:01<00:02, 103.76it/s]




 36%|███▌      | 113/313 [00:01<00:01, 106.32it/s]




 40%|███▉      | 124/313 [00:01<00:01, 106.74it/s]




 43%|████▎     | 135/313 [00:01<00:01, 104.60it/s]




 47%|████▋     | 146/313 [00:01<00:01, 105.82it/s]




 50%|█████     | 157/313 [00:01<00:01, 93.22it/s] 




 53%|█████▎    | 167/313 [00:01<00:01, 88.01it/s]




 57%|█████▋    | 178/313 [00:01<00:01, 92.50it/s]




 60%|██████    | 188/313 [00:01<00:01

Epoch: 9; Accuracy train: 98.760%


### RMSProp  Accuracy train: 96.440% 
### SGD Accuracy train = 98.760%